In [1]:
from data.dataset import Dataset
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_ = Dataset()
dataset = data_.prepare()

READING DATASET
PREPARING DATASET
	 cleaned null values...
	 extracted datetime variables...


In [ ]:
dataset.head()